In [ ]:
from sympy import *
from dmdgp_ML import *
import plotly.express as px
import numpy as np
import os

# Missing Data

In [ ]:
wdir = '/home/michael/gitrepos/dmdgp-mlm/DATA_D4_N100_S100'
fn = os.path.join(wdir, 'pid_0000.nmr')
D = read_nmr(fn)
n = np.max(list(D.keys()))
PID, Y = [], []
FN = filter(lambda fn: fn.endswith('.nmr'), os.listdir(wdir))
for fn in sorted(FN):    
    pid = int(fn.replace('.nmr', '').split('_')[1])
    fn = os.path.join(wdir, fn)
    D = read_nmr(fn)
    x, y = [i for i in range(4, n)], []
    for i in x:
        # only predecessors
        S = [j for j in D[i] if j < i]        
        y.append(len(S))
    y = [v for v in y if v < 4]
    Y.append(len(y))
    PID.append(pid) 
df = {'NUM OF NODES WITHOUT 4 PREDECESSORS':Y, 'PID':PID}
px.scatter(df,x='PID', y='NUM OF NODES WITHOUT 4 PREDECESSORS')


In [ ]:
pid = 23
fn = os.path.join(wdir, 'pid_%04d.nmr' % pid)
D = read_nmr(fn)
x, y, s = [], [], []
for i in [i for i in range(4, n)]:
    # only predecessors
    S = [j for j in D[i] if j < i]    
    x.append(i)
    y.append(len(S))
    s.append(len(S) >= 4)
df = {'atom id':x,'#preds':y, '#preds >= 4':s}
px.scatter(df, x='atom id', y='#preds', color='#preds >= 4', symbol='#preds >= 4')

# Symbolic calculations

## Naive Model

$$f(x)= \sum_{(i,j)\in E} \left(d_{ij}^2-\sum_{k=1}^3(x_{ik}-x_{jk})^2\right)^2$$

In [ ]:
x = {}
for i in range(5):
    for j in range(3):
        x[i,j] = symbols('x_%d_%d' % (i,j))
x

In [ ]:
D = {}

In [ ]:
E = read_nmr('/home/michael/gitrepos/dmdgp-mlm/DATA_N5_S5/pid_0000.nmr')
y = read_csv('/home/michael/gitrepos/dmdgp-mlm/DATA_N5_S5/pid_0001.csv')


In [ ]:
D = {}
for i in sorted(E):
    D[i] = {}
    for j in sorted(E[i]):
        if j > i:
            break
        D[i][j] = symbols('D_%d_%d' % (i,j))
D

In [ ]:
def funF(D, x):
    f = 0
    for i in sorted(D):
        for j in sorted(D[i]):
            fij = (x[i, 0] - x[j, 0])**2 + (x[i, 1] - x[j, 1])**2 + (x[i, 2] - x[j, 2])**2
            fij = fij - D[i][j]**2
            f += fij**2
    return f


In [ ]:
f = funF(D, x)
f

In [ ]:
g = [f.diff(x[ij]) for ij in x]
g

In [ ]:
subs={x[ij]:y[ij] for ij in x}
for i in D:
    for j in D[i]:
        subs[D[i][j]] = E[i][j]
f.evalf(subs=subs)

In [ ]:
G=[gij.evalf(subs=subs) for gij in g]
G
